In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "Qwen/Qwen2.5-1.5b-Instruct"

config_8bit = BitsAndBytesConfig(load_in_8bit=True)

model_8bit = AutoModelForCausalLM.from_pretrained(

                                                  model_name,
                                                  quantization_config=config_8bit,
                                                  trust_remote_code=True,
                                                  device_map="auto",
                                                  )

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [9]:
from datasets import load_dataset

dataset = load_dataset("HumanLLMs/Human-Like-DPO-Dataset",split="train[:300]")
dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 300
})

In [10]:
dataset[0]

{'prompt': 'Oh, I just saw the best meme - have you seen it?',
 'chosen': "😂 Ah, no I haven't! I'm dying to know, what's the meme about? Is it a funny cat or a ridiculous situation? Spill the beans! 🤣",
 'rejected': "I'm an artificial intelligence language model, I don't have personal experiences or opinions. However, I can provide you with information on highly-rated and critically acclaimed films, as well as recommendations based on specific genres or themes. Would you like me to suggest some notable movies or discuss a particular genre of interest?"}

In [11]:
example_text = [{'role':'assistant','content':dataset[0]['chosen']}]
example_text

[{'role': 'assistant',
  'content': "😂 Ah, no I haven't! I'm dying to know, what's the meme about? Is it a funny cat or a ridiculous situation? Spill the beans! 🤣"}]

In [12]:
tokenizer.apply_chat_template(example_text,tokenizer=False)

[151644,
 8948,
 198,
 2610,
 525,
 1207,
 16948,
 11,
 3465,
 553,
 54364,
 14817,
 13,
 1446,
 525,
 264,
 10950,
 17847,
 13,
 151645,
 198,
 151644,
 77091,
 198,
 144185,
 16366,
 11,
 902,
 358,
 8990,
 944,
 0,
 358,
 2776,
 22273,
 311,
 1414,
 11,
 1128,
 594,
 279,
 41185,
 911,
 30,
 2160,
 432,
 264,
 15173,
 8251,
 476,
 264,
 26775,
 6534,
 30,
 3089,
 483,
 279,
 26896,
 0,
 11162,
 97,
 96,
 151645,
 198]

In [13]:
def apply_chat_temp(example):

    example['prompt'] =[{'role':'user','content':example['prompt']}]

    return example


new_dataset = dataset.map(apply_chat_temp)
new_dataset[0]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

{'prompt': [{'content': 'Oh, I just saw the best meme - have you seen it?',
   'role': 'user'}],
 'chosen': "😂 Ah, no I haven't! I'm dying to know, what's the meme about? Is it a funny cat or a ridiculous situation? Spill the beans! 🤣",
 'rejected': "I'm an artificial intelligence language model, I don't have personal experiences or opinions. However, I can provide you with information on highly-rated and critically acclaimed films, as well as recommendations based on specific genres or themes. Would you like me to suggest some notable movies or discuss a particular genre of interest?"}

In [14]:
new_templete = """<|im_start|>assistant\n{model_answer}<|im_end|>\n"""


def format_dataset(example):

    example['prompt'] = tokenizer.apply_chat_template(example['prompt'],tokenize=False)
    example['chosen'] = new_templete.format(model_answer =example['chosen'])
    example['rejected'] = new_templete.format(model_answer =example['rejected'])

    return example


formatted_dataset = new_dataset.map(format_dataset)
formatted_dataset[0]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

{'prompt': '<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nOh, I just saw the best meme - have you seen it?<|im_end|>\n',
 'chosen': "<|im_start|>assistant\n😂 Ah, no I haven't! I'm dying to know, what's the meme about? Is it a funny cat or a ridiculous situation? Spill the beans! 🤣<|im_end|>\n",
 'rejected': "<|im_start|>assistant\nI'm an artificial intelligence language model, I don't have personal experiences or opinions. However, I can provide you with information on highly-rated and critically acclaimed films, as well as recommendations based on specific genres or themes. Would you like me to suggest some notable movies or discuss a particular genre of interest?<|im_end|>\n"}

In [15]:
model_8bit

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear8bitLt(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear8bitLt(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear8bitLt(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear8bitLt(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear8bitLt(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear8bitLt(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear8bitLt(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNor

In [16]:
from peft import LoraConfig, get_peft_model

lora_config= LoraConfig(

                        r = 32,
                        lora_alpha = 32,
                        target_modules =["q_proj","k_proj","v_proj","o_proj",
                  "gate_proj","up_proj","down_proj"],
                        lora_dropout = 0.05,
                        bias = "none",
                        task_type = "CAUSAL_LM"
)
lora_model = get_peft_model(model_8bit,lora_config)
lora_model.print_trainable_parameters()

trainable params: 36,929,536 || all params: 1,580,643,840 || trainable%: 2.3364


In [17]:
lora_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): 

In [18]:
from trl import DPOTrainer, DPOConfig

dpo_config = DPOConfig(

                      output_dir = "./results",
                      per_device_train_batch_size = 2,
                      max_steps = 40,
                      gradient_accumulation_steps = 4,
                      learning_rate = 3e-5,
                      lr_scheduler_type ="cosine",
                      optim = "adamw_torch",
                      logging_steps = 10,
                      beta = 0.01,
                      report_to = "none"

)

In [22]:
trainer = DPOTrainer(

                     model = lora_model,
                     train_dataset = formatted_dataset,
                    args = dpo_config,
                    processing_class=tokenizer
)

trainer.train()

Extracting prompt in train dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
c:\Users\abutair\workspace\nn-zero-to-hero\.conda\lib\site-packages\transformers\integrations\sdpa_attention.py:54: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Step,Training Loss
10,0.543300
20,0.140200
30,0.017800
40,0.011600


TrainOutput(global_step=40, training_loss=0.17825579643249512, metrics={'train_runtime': 112.4546, 'train_samples_per_second': 2.846, 'train_steps_per_second': 0.356, 'total_flos': 0.0, 'train_loss': 0.17825579643249512, 'epoch': 1.0533333333333332})

In [23]:
lora_path = "./lora_adapters"
lora_model.save_pretrained(lora_path)

In [24]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
lora_path = "./lora_adapters"


name_name = "Qwen/Qwen2.5-1.5B-Instruct"




base_model = AutoModelForCausalLM.from_pretrained(

                                                  name_name,

                                                  trust_remote_code=True,
                                                  device_map="auto",
                                                  )

tokenizer = AutoTokenizer.from_pretrained(name_name, trust_remote_code=True)

In [25]:
from peft import PeftModel

final_model = PeftModel.from_pretrained(base_model,lora_path)
final_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(

In [26]:
final_model = final_model.merge_and_unload()
final_model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw